# A Fundamental Example of Machine Learning
*by James Medaugh*

## Overview
The goal of this notebook is to demonstrate one of the most elementary possible examples of a machine learning implementation. In the interest of simplicity, the model will use linear regression to estimate one output variable based upon the value of one numeric input variable.  I will use a few well known libraries such as matplotlib for graphing, pandas for reading csv data, and few others, but the focus of this notebook should be the mathematics and visual representations.

## Scenario
The data we will use for this is a simple and silly example.  The only input variable, also called a feature, will be "number of explosions".  The output variable will be "James' rating of the film" with values ranging from 0 to 10.  The dataset contains records 50 fictious films.  It is also worth mentioning that the movie rating precision is egregiously specific (e.g. 6.025596752 movie rating out of 10).  Pretend for the sake of the example that I actually care to be that precise about how much I like a movie.

    Independent Variable (X) - # of explosions in a film
    Dependent Variable (Y) - James' rating of film
    Model - Linear Regression
    Optimization Method - Batch Gradient Descent

## Steps

First thing we need to do is get the data out of the CSV and into a python object.  Here I'm using Pandas to read the CSV file and sorting the data once it is read so it is easier to plot

In [1]:
import pandas as pd

data = pd.read_csv('movie_ratings.csv')
data = data.sort_values(by=['num_explosions'])

Next, we are going to use matplotlib to create a scatter plot.  We are graphing our X variable, number of explosions, against our Y variable James' movie rating.  I also added some axis scaling and labels.  Let's see if any patterns emerge...

In [2]:
from matplotlib import pyplot as plt

plt.scatter(data['num_explosions'],data['james_movie_rating'])
plt.axis([0, 10, 0, 10])
plt.xlabel('Number of Explosions')
plt.ylabel('James'' movie rating' )
plt.title('Scatter Plot of Movie Data')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

We can see there is a pretty obvious linear relationship here.  Our next goal is to draw a line that best matches the trend displayed on the graph.  This line has many names: regression line, line of best fit, trendline and perhaps others.  But while the line may have many names, it only has one universal equation:

![Line Equation](https://raw.githubusercontent.com/jaymztheking/Machine_Learning_Primer/master/images/SlopeEq.gif)

This becomes our model.  Our X variable of number of explosions times some value m plus some value b will let us estimate the movie rating.  But how do we find the correct values for m and b?  There are an infinte combination of values for m and b together that define an infinite number.  Enter the cost function.
    

## Cost Function - Scoring Your Model

When determining the regression line purely from a visual standpoint, it becomes clear that not all lines are created equal...

For example this line (m=.7, b= 3) is fairly close to most of the data points


In [ ]:
import numpy as np

#Defines the trend line
m = .7
b = 3
linex = np.arange(0,11)        #list of number 0 to 10
liney = m*linex+b              #line equation

#Scatter Plot and Formatting
plt.scatter(data['num_explosions'],data['james_movie_rating'])
plt.axis([0, 10, 0, 10])
plt.xlabel('Number of Explosions')
plt.ylabel('James'' movie rating' )
plt.title('Scatter Plot of Movie Data')

#Adds trend line
plt.plot(linex,liney)
plt.show()

But this line (m= 10, b= -20) is no where close to matching all of the data points

In [ ]:
import numpy as np

#Defines the trend line
m = 10
b = -20
linex = np.arange(0,11)        #list of number 0 to 10
liney = m*linex+b              #line equation

#Scatter Plot and Formatting
plt.scatter(data['num_explosions'],data['james_movie_rating'])
plt.axis([0, 10, 0, 10])
plt.xlabel('Number of Explosions')
plt.ylabel('James'' movie rating' )
plt.title('Scatter Plot of Movie Data')

#Adds trend line
plt.plot(linex,liney)
plt.show()

We know intuitively why the first example is a good fit and the second example is a bad fit.  However we need a way to empirically measure just how close or far the line is to the data points to quantify just how good or bad of a fit a line is.  This is accomplished by calculating the mean squared error.  The equation to calculate mean squared error is
    ![MSE Formula](data:image/svg+xml;base64,PHN2ZyB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgd2lkdGg9IjI0%0D%0ALjcyOWV4IiBoZWlnaHQ9IjYuODQzZXgiIHN0eWxlPSJ2ZXJ0aWNhbC1hbGlnbjogLTMuMDA1ZXg7%0D%0AIiB2aWV3Qm94PSIwIC0xNjUyLjUgMTA2NDcuMiAyOTQ2LjEiIHJvbGU9ImltZyIgZm9jdXNhYmxl%0D%0APSJmYWxzZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiBhcmlhLWxhYmVsbGVk%0D%0AYnk9Ik1hdGhKYXgtU1ZHLTEtVGl0bGUiPgo8dGl0bGUgaWQ9Ik1hdGhKYXgtU1ZHLTEtVGl0bGUi%0D%0APntcZGlzcGxheXN0eWxlIFxvcGVyYXRvcm5hbWUge01TRX0gPXtcZnJhYyB7MX17bn19XHN1bSBf%0D%0Ae2k9MX1ee259KFlfe2l9LXtcaGF0IHtZX3tpfX19KV57Mn0ufTwvdGl0bGU+CjxkZWZzIGFyaWEt%0D%0AaGlkZGVuPSJ0cnVlIj4KPHBhdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0iRTEtTUpNQUlOLTREIiBk%0D%0APSJNMTMyIDYyMlExMjUgNjI5IDEyMSA2MzFUMTA1IDYzNFQ2MiA2MzdIMjlWNjgzSDEzNVEyMjEg%0D%0ANjgzIDIzMiA2ODJUMjQ5IDY3NVEyNTAgNjc0IDM1NCAzOThMNDU4IDEyNEw1NjIgMzk4UTY2NiA2%0D%0ANzQgNjY4IDY3NVE2NzEgNjgxIDY4MyA2ODJUNzgxIDY4M0g4ODdWNjM3SDg1NFE4MTQgNjM2IDgw%0D%0AMyA2MzRUNzg1IDYyMlY2MVE3OTEgNTEgODAyIDQ5VDg1NCA0Nkg4ODdWMEg4NzZRODU1IDMgNzM2%0D%0AIDNRNjA1IDMgNTk2IDBINTg1VjQ2SDYxOFE2NjAgNDcgNjY5IDQ5VDY4OCA2MVYzNDdRNjg4IDQy%0D%0ANCA2ODggNDYxVDY4OCA1NDZUNjg4IDYxM0w2ODcgNjMyUTQ1NCAxNCA0NTAgN1E0NDYgMSA0MzAg%0D%0AMVQ0MTAgN1E0MDkgOSAyOTIgMzE2TDE3NiA2MjRWNjA2UTE3NSA1ODggMTc1IDU0M1QxNzUgNDYz%0D%0AVDE3NSAzNTZMMTc2IDg2UTE4NyA1MCAyNjEgNDZIMjc4VjBIMjY5UTI1NCAzIDE1NCAzUTUyIDMg%0D%0AMzcgMEgyOVY0Nkg0NlE3OCA0OCA5OCA1NlQxMjIgNjlUMTMyIDg2VjYyMloiPjwvcGF0aD4KPHBh%0D%0AdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0iRTEtTUpNQUlOLTUzIiBkPSJNNTUgNTA3UTU1IDU5MCAx%0D%0AMTIgNjQ3VDI0MyA3MDRIMjU3UTM0MiA3MDQgNDA1IDY0MUw0MjYgNjcyUTQzMSA2NzkgNDM2IDY4%0D%0AN1Q0NDYgNzAwTDQ0OSA3MDRRNDUwIDcwNCA0NTMgNzA0VDQ1OSA3MDVINDYzUTQ2NiA3MDUgNDcy%0D%0AIDY5OVY0NjJMNDY2IDQ1Nkg0NDhRNDM3IDQ1NiA0MzUgNDU5VDQzMCA0NzlRNDEzIDYwNSAzMjkg%0D%0ANjQ2UTI5MiA2NjIgMjU0IDY2MlEyMDEgNjYyIDE2OCA2MjZUMTM1IDU0MlExMzUgNTA4IDE1MiA0%0D%0AODBUMjAwIDQzNVEyMTAgNDMxIDI4NiA0MTJUMzcwIDM4OVE0MjcgMzY3IDQ2MyAzMTRUNTAwIDE5%0D%0AMVE1MDAgMTEwIDQ0OCA0NVQzMDEgLTIxUTI0NSAtMjEgMjAxIC00VDE0MCAyN0wxMjIgNDFRMTE4%0D%0AIDM2IDEwNyAyMVQ4NyAtN1Q3OCAtMjFRNzYgLTIyIDY4IC0yMkg2NFE2MSAtMjIgNTUgLTE2VjEw%0D%0AMVE1NSAyMjAgNTYgMjIyUTU4IDIyNyA3NiAyMjdIODlROTUgMjIxIDk1IDIxNFE5NSAxODIgMTA1%0D%0AIDE1MVQxMzkgOTBUMjA1IDQyVDMwNSAyNFEzNTIgMjQgMzg2IDYyVDQyMCAxNTVRNDIwIDE5OCAz%0D%0AOTggMjMzVDM0MCAyODFRMjg0IDI5NSAyNjYgMzAwUTI2MSAzMDEgMjM5IDMwNlQyMDYgMzE0VDE3%0D%0ANCAzMjVUMTQxIDM0M1QxMTIgMzY3VDg1IDQwMlE1NSA0NTEgNTUgNTA3WiI+PC9wYXRoPgo8cGF0%0D%0AaCBzdHJva2Utd2lkdGg9IjEiIGlkPSJFMS1NSk1BSU4tNDUiIGQ9Ik0xMjggNjE5UTEyMSA2MjYg%0D%0AMTE3IDYyOFQxMDEgNjMxVDU4IDYzNEgyNVY2ODBINTk3VjY3NlE1OTkgNjcwIDYxMSA1NjBUNjI1%0D%0AIDQ0NFY0NDBINTg1VjQ0NFE1ODQgNDQ3IDU4MiA0NjVRNTc4IDUwMCA1NzAgNTI2VDU1MyA1NzFU%0D%0ANTI4IDYwMVQ0OTggNjE5VDQ1NyA2MjlUNDExIDYzM1QzNTMgNjM0UTI2NiA2MzQgMjUxIDYzM1Qy%0D%0AMzMgNjIyUTIzMyA2MjIgMjMzIDYyMVEyMzIgNjE5IDIzMiA0OTdWMzc2SDI4NlEzNTkgMzc4IDM3%0D%0ANyAzODVRNDEzIDQwMSA0MTYgNDY5UTQxNiA0NzEgNDE2IDQ3M1Y0OTNINDU2VjIxM0g0MTZWMjMz%0D%0AUTQxNSAyNjggNDA4IDI4OFQzODMgMzE3VDM0OSAzMjhUMjk3IDMzMFEyOTAgMzMwIDI4NiAzMzBI%0D%0AMjMyVjE5NlYxMTRRMjMyIDU3IDIzNyA1MlEyNDMgNDcgMjg5IDQ3SDM0MEgzOTFRNDI4IDQ3IDQ1%0D%0AMiA1MFQ1MDUgNjJUNTUyIDkyVDU4NCAxNDZRNTk0IDE3MiA1OTkgMjAwVDYwNyAyNDdUNjEyIDI3%0D%0AMFYyNzNINjUyVjI3MFE2NTEgMjY3IDYzMiAxMzdUNjEwIDNWMEgyNVY0Nkg1OFExMDAgNDcgMTA5%0D%0AIDQ5VDEyOCA2MVY2MTlaIj48L3BhdGg+CjxwYXRoIHN0cm9rZS13aWR0aD0iMSIgaWQ9IkUxLU1K%0D%0ATUFJTi0zRCIgZD0iTTU2IDM0N1E1NiAzNjAgNzAgMzY3SDcwN1E3MjIgMzU5IDcyMiAzNDdRNzIy%0D%0AIDMzNiA3MDggMzI4TDM5MCAzMjdINzJRNTYgMzMyIDU2IDM0N1pNNTYgMTUzUTU2IDE2OCA3MiAx%0D%0ANzNINzA4UTcyMiAxNjMgNzIyIDE1M1E3MjIgMTQwIDcwNyAxMzNINzBRNTYgMTQwIDU2IDE1M1oi%0D%0APjwvcGF0aD4KPHBhdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0iRTEtTUpNQUlOLTMxIiBkPSJNMjEz%0D%0AIDU3OEwyMDAgNTczUTE4NiA1NjggMTYwIDU2M1QxMDIgNTU2SDgzVjYwMkgxMDJRMTQ5IDYwNCAx%0D%0AODkgNjE3VDI0NSA2NDFUMjczIDY2M1EyNzUgNjY2IDI4NSA2NjZRMjk0IDY2NiAzMDIgNjYwVjM2%0D%0AMUwzMDMgNjFRMzEwIDU0IDMxNSA1MlQzMzkgNDhUNDAxIDQ2SDQyN1YwSDQxNlEzOTUgMyAyNTcg%0D%0AM1ExMjEgMyAxMDAgMEg4OFY0NkgxMTRRMTM2IDQ2IDE1MiA0NlQxNzcgNDdUMTkzIDUwVDIwMSA1%0D%0AMlQyMDcgNTdUMjEzIDYxVjU3OFoiPjwvcGF0aD4KPHBhdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0i%0D%0ARTEtTUpNQVRISS02RSIgZD0iTTIxIDI4N1EyMiAyOTMgMjQgMzAzVDM2IDM0MVQ1NiAzODhUODkg%0D%0ANDI1VDEzNSA0NDJRMTcxIDQ0MiAxOTUgNDI0VDIyNSAzOTBUMjMxIDM2OVEyMzEgMzY3IDIzMiAz%0D%0ANjdMMjQzIDM3OFEzMDQgNDQyIDM4MiA0NDJRNDM2IDQ0MiA0NjkgNDE1VDUwMyAzMzZUNDY1IDE3%0D%0AOVQ0MjcgNTJRNDI3IDI2IDQ0NCAyNlE0NTAgMjYgNDUzIDI3UTQ4MiAzMiA1MDUgNjVUNTQwIDE0%0D%0ANVE1NDIgMTUzIDU2MCAxNTNRNTgwIDE1MyA1ODAgMTQ1UTU4MCAxNDQgNTc2IDEzMFE1NjggMTAx%0D%0AIDU1NCA3M1Q1MDggMTdUNDM5IC0xMFEzOTIgLTEwIDM3MSAxN1QzNTAgNzNRMzUwIDkyIDM4NiAx%0D%0AOTNUNDIzIDM0NVE0MjMgNDA0IDM3OSA0MDRIMzc0UTI4OCA0MDQgMjI5IDMwM0wyMjIgMjkxTDE4%0D%0AOSAxNTdRMTU2IDI2IDE1MSAxNlExMzggLTExIDEwOCAtMTFROTUgLTExIDg3IC01VDc2IDdUNzQg%0D%0AMTdRNzQgMzAgMTEyIDE4MFQxNTIgMzQzUTE1MyAzNDggMTUzIDM2NlExNTMgNDA1IDEyOSA0MDVR%0D%0AOTEgNDA1IDY2IDMwNVE2MCAyODUgNjAgMjg0UTU4IDI3OCA0MSAyNzhIMjdRMjEgMjg0IDIxIDI4%0D%0AN1oiPjwvcGF0aD4KPHBhdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0iRTEtTUpTWjItMjIxMSIgZD0i%0D%0ATTYwIDk0OFE2MyA5NTAgNjY1IDk1MEgxMjY3TDEzMjUgODE1UTEzODQgNjc3IDEzODggNjY5SDEz%0D%0ANDhMMTM0MSA2ODNRMTMyMCA3MjQgMTI4NSA3NjFRMTIzNSA4MDkgMTE3NCA4MzhUMTAzMyA4ODFU%0D%0AODgyIDg5OFQ2OTkgOTAySDU3NEg1NDNIMjUxTDI1OSA4OTFRNzIyIDI1OCA3MjQgMjUyUTcyNSAy%0D%0ANTAgNzI0IDI0NlE3MjEgMjQzIDQ2MCAtNTZMMTk2IC0zNTZRMTk2IC0zNTcgNDA3IC0zNTdRNDU5%0D%0AIC0zNTcgNTQ4IC0zNTdUNjc2IC0zNThRODEyIC0zNTggODk2IC0zNTNUMTA2MyAtMzMyVDEyMDQg%0D%0ALTI4M1QxMzA3IC0xOTZRMTMyOCAtMTcwIDEzNDggLTEyNEgxMzg4UTEzODggLTEyNSAxMzgxIC0x%0D%0ANDVUMTM1NiAtMjEwVDEzMjUgLTI5NEwxMjY3IC00NDlMNjY2IC00NTBRNjQgLTQ1MCA2MSAtNDQ4%0D%0AUTU1IC00NDYgNTUgLTQzOVE1NSAtNDM3IDU3IC00MzNMNTkwIDE3N1E1OTAgMTc4IDU1NyAyMjJU%0D%0ANDUyIDM2NlQzMjIgNTQ0TDU2IDkwOUw1NSA5MjRRNTUgOTQ1IDYwIDk0OFoiPjwvcGF0aD4KPHBh%0D%0AdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0iRTEtTUpNQVRISS02OSIgZD0iTTE4NCA2MDBRMTg0IDYy%0D%0ANCAyMDMgNjQyVDI0NyA2NjFRMjY1IDY2MSAyNzcgNjQ5VDI5MCA2MTlRMjkwIDU5NiAyNzAgNTc3%0D%0AVDIyNiA1NTdRMjExIDU1NyAxOTggNTY3VDE4NCA2MDBaTTIxIDI4N1EyMSAyOTUgMzAgMzE4VDU0%0D%0AIDM2OVQ5OCA0MjBUMTU4IDQ0MlExOTcgNDQyIDIyMyA0MTlUMjUwIDM1N1EyNTAgMzQwIDIzNiAz%0D%0AMDFUMTk2IDE5NlQxNTQgODNRMTQ5IDYxIDE0OSA1MVExNDkgMjYgMTY2IDI2UTE3NSAyNiAxODUg%0D%0AMjlUMjA4IDQzVDIzNSA3OFQyNjAgMTM3UTI2MyAxNDkgMjY1IDE1MVQyODIgMTUzUTMwMiAxNTMg%0D%0AMzAyIDE0M1EzMDIgMTM1IDI5MyAxMTJUMjY4IDYxVDIyMyAxMVQxNjEgLTExUTEyOSAtMTEgMTAy%0D%0AIDEwVDc0IDc0UTc0IDkxIDc5IDEwNlQxMjIgMjIwUTE2MCAzMjEgMTY2IDM0MVQxNzMgMzgwUTE3%0D%0AMyA0MDQgMTU2IDQwNEgxNTRRMTI0IDQwNCA5OSAzNzFUNjEgMjg3UTYwIDI4NiA1OSAyODRUNTgg%0D%0AMjgxVDU2IDI3OVQ1MyAyNzhUNDkgMjc4VDQxIDI3OEgyN1EyMSAyODQgMjEgMjg3WiI+PC9wYXRo%0D%0APgo8cGF0aCBzdHJva2Utd2lkdGg9IjEiIGlkPSJFMS1NSk1BSU4tMjgiIGQ9Ik05NCAyNTBROTQg%0D%0AMzE5IDEwNCAzODFUMTI3IDQ4OFQxNjQgNTc2VDIwMiA2NDNUMjQ0IDY5NVQyNzcgNzI5VDMwMiA3%0D%0ANTBIMzE1SDMxOVEzMzMgNzUwIDMzMyA3NDFRMzMzIDczOCAzMTYgNzIwVDI3NSA2NjdUMjI2IDU4%0D%0AMVQxODQgNDQzVDE2NyAyNTBUMTg0IDU4VDIyNSAtODFUMjc0IC0xNjdUMzE2IC0yMjBUMzMzIC0y%0D%0ANDFRMzMzIC0yNTAgMzE4IC0yNTBIMzE1SDMwMkwyNzQgLTIyNlExODAgLTE0MSAxMzcgLTE0VDk0%0D%0AIDI1MFoiPjwvcGF0aD4KPHBhdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0iRTEtTUpNQVRISS01OSIg%0D%0AZD0iTTY2IDYzN1E1NCA2MzcgNDkgNjM3VDM5IDYzOFQzMiA2NDFUMzAgNjQ3VDMzIDY2NFQ0MiA2%0D%0AODJRNDQgNjgzIDU2IDY4M1ExMDQgNjgwIDE2NSA2ODBRMjg4IDY4MCAzMDYgNjgzSDMxNlEzMjIg%0D%0ANjc3IDMyMiA2NzRUMzIwIDY1NlEzMTYgNjQzIDMxMCA2MzdIMjk4UTI0MiA2MzcgMjQyIDYyNFEy%0D%0ANDIgNjE5IDI5MiA0NzdUMzQzIDMzM0wzNDYgMzM2UTM1MCAzNDAgMzU4IDM0OVQzNzkgMzczVDQx%0D%0AMSA0MTBUNDU0IDQ2MVE1NDYgNTY4IDU2MSA1ODdUNTc3IDYxOFE1NzcgNjM0IDU0NSA2MzdRNTI4%0D%0AIDYzNyA1MjggNjQ3UTUyOCA2NDkgNTMwIDY2MVE1MzMgNjc2IDUzNSA2NzlUNTQ5IDY4M1E1NTEg%0D%0ANjgzIDU3OCA2ODJUNjU3IDY4MFE2ODQgNjgwIDcxMyA2ODFUNzQ2IDY4MlE3NjMgNjgyIDc2MyA2%0D%0ANzNRNzYzIDY2OSA3NjAgNjU3VDc1NSA2NDNRNzUzIDYzNyA3MzQgNjM3UTY2MiA2MzIgNjE3IDU4%0D%0AN1E2MDggNTc4IDQ3NyA0MjRMMzQ4IDI3M0wzMjIgMTY5UTI5NSA2MiAyOTUgNTdRMjk1IDQ2IDM2%0D%0AMyA0NlEzNzkgNDYgMzg0IDQ1VDM5MCAzNVEzOTAgMzMgMzg4IDIzUTM4NCA2IDM4MiA0VDM2NiAx%0D%0AUTM2MSAxIDMyNCAxVDIzMiAyUTE3MCAyIDEzOCAyVDEwMiAxUTg0IDEgODQgOVE4NCAxNCA4NyAy%0D%0ANFE4OCAyNyA4OSAzMFQ5MCAzNVQ5MSAzOVQ5MyA0MlQ5NiA0NFQxMDEgNDVUMTA3IDQ1VDExNiA0%0D%0ANlQxMjkgNDZRMTY4IDQ3IDE4MCA1MFQxOTggNjNRMjAxIDY4IDIyNyAxNzFMMjUyIDI3NEwxMjkg%0D%0ANjIzUTEyOCA2MjQgMTI3IDYyNVQxMjUgNjI3VDEyMiA2MjlUMTE4IDYzMVQxMTMgNjMzVDEwNSA2%0D%0AMzRUOTYgNjM1VDgzIDYzNlQ2NiA2MzdaIj48L3BhdGg+CjxwYXRoIHN0cm9rZS13aWR0aD0iMSIg%0D%0AaWQ9IkUxLU1KTUFJTi0yMjEyIiBkPSJNODQgMjM3VDg0IDI1MFQ5OCAyNzBINjc5UTY5NCAyNjIg%0D%0ANjk0IDI1MFQ2NzkgMjMwSDk4UTg0IDIzNyA4NCAyNTBaIj48L3BhdGg+CjxwYXRoIHN0cm9rZS13%0D%0AaWR0aD0iMSIgaWQ9IkUxLU1KTUFJTi01RSIgZD0iTTExMiA1NjBMMjQ5IDY5NEwyNTcgNjg2UTM4%0D%0ANyA1NjIgMzg3IDU2MEwzNjEgNTMxUTM1OSA1MzIgMzAzIDU4MUwyNTAgNjI3TDE5NSA1ODBRMTgy%0D%0AIDU2OSAxNjkgNTU3VDE0OCA1MzhMMTQwIDUzMlExMzggNTMwIDEyNSA1NDZMMTEyIDU2MFoiPjwv%0D%0AcGF0aD4KPHBhdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0iRTEtTUpNQUlOLTI5IiBkPSJNNjAgNzQ5%0D%0ATDY0IDc1MFE2OSA3NTAgNzQgNzUwSDg2TDExNCA3MjZRMjA4IDY0MSAyNTEgNTE0VDI5NCAyNTBR%0D%0AMjk0IDE4MiAyODQgMTE5VDI2MSAxMlQyMjQgLTc2VDE4NiAtMTQzVDE0NSAtMTk0VDExMyAtMjI3%0D%0AVDkwIC0yNDZRODcgLTI0OSA4NiAtMjUwSDc0UTY2IC0yNTAgNjMgLTI1MFQ1OCAtMjQ3VDU1IC0y%0D%0AMzhRNTYgLTIzNyA2NiAtMjI1UTIyMSAtNjQgMjIxIDI1MFQ2NiA3MjVRNTYgNzM3IDU1IDczOFE1%0D%0ANSA3NDYgNjAgNzQ5WiI+PC9wYXRoPgo8cGF0aCBzdHJva2Utd2lkdGg9IjEiIGlkPSJFMS1NSk1B%0D%0ASU4tMzIiIGQ9Ik0xMDkgNDI5UTgyIDQyOSA2NiA0NDdUNTAgNDkxUTUwIDU2MiAxMDMgNjE0VDIz%0D%0ANSA2NjZRMzI2IDY2NiAzODcgNjEwVDQ0OSA0NjVRNDQ5IDQyMiA0MjkgMzgzVDM4MSAzMTVUMzAx%0D%0AIDI0MVEyNjUgMjEwIDIwMSAxNDlMMTQyIDkzTDIxOCA5MlEzNzUgOTIgMzg1IDk3UTM5MiA5OSA0%0D%0AMDkgMTg2VjE4OUg0NDlWMTg2UTQ0OCAxODMgNDM2IDk1VDQyMSAzVjBINTBWMTlWMzFRNTAgMzgg%0D%0ANTYgNDZUODYgODFRMTE1IDExMyAxMzYgMTM3UTE0NSAxNDcgMTcwIDE3NFQyMDQgMjExVDIzMyAy%0D%0ANDRUMjYxIDI3OFQyODQgMzA4VDMwNSAzNDBUMzIwIDM2OVQzMzMgNDAxVDM0MCA0MzFUMzQzIDQ2%0D%0ANFEzNDMgNTI3IDMwOSA1NzNUMjEyIDYxOVExNzkgNjE5IDE1NCA2MDJUMTE5IDU2OVQxMDkgNTUw%0D%0AUTEwOSA1NDkgMTE0IDU0OVExMzIgNTQ5IDE1MSA1MzVUMTcwIDQ4OVExNzAgNDY0IDE1NCA0NDdU%0D%0AMTA5IDQyOVoiPjwvcGF0aD4KPHBhdGggc3Ryb2tlLXdpZHRoPSIxIiBpZD0iRTEtTUpNQUlOLTJF%0D%0AIiBkPSJNNzggNjBRNzggODQgOTUgMTAyVDEzOCAxMjBRMTYyIDEyMCAxODAgMTA0VDE5OSA2MVEx%0D%0AOTkgMzYgMTgyIDE4VDEzOSAwVDk2IDE3VDc4IDYwWiI+PC9wYXRoPgo8L2RlZnM+CjxnIHN0cm9r%0D%0AZT0iY3VycmVudENvbG9yIiBmaWxsPSJjdXJyZW50Q29sb3IiIHN0cm9rZS13aWR0aD0iMCIgdHJh%0D%0AbnNmb3JtPSJtYXRyaXgoMSAwIDAgLTEgMCAwKSIgYXJpYS1oaWRkZW49InRydWUiPgogPHVzZSB4%0D%0AbGluazpocmVmPSIjRTEtTUpNQUlOLTREIj48L3VzZT4KIDx1c2UgeGxpbms6aHJlZj0iI0UxLU1K%0D%0ATUFJTi01MyIgeD0iOTE3IiB5PSIwIj48L3VzZT4KIDx1c2UgeGxpbms6aHJlZj0iI0UxLU1KTUFJ%0D%0ATi00NSIgeD0iMTQ3NCIgeT0iMCI+PC91c2U+CiA8dXNlIHhsaW5rOmhyZWY9IiNFMS1NSk1BSU4t%0D%0AM0QiIHg9IjI0MzMiIHk9IjAiPjwvdXNlPgo8ZyB0cmFuc2Zvcm09InRyYW5zbGF0ZSgzNDg5LDAp%0D%0AIj4KPGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMTIwLDApIj4KPHJlY3Qgc3Ryb2tlPSJub25lIiB3%0D%0AaWR0aD0iNzIwIiBoZWlnaHQ9IjYwIiB4PSIwIiB5PSIyMjAiPjwvcmVjdD4KIDx1c2UgeGxpbms6%0D%0AaHJlZj0iI0UxLU1KTUFJTi0zMSIgeD0iMTEwIiB5PSI2NzYiPjwvdXNlPgogPHVzZSB4bGluazpo%0D%0AcmVmPSIjRTEtTUpNQVRISS02RSIgeD0iNjAiIHk9Ii02ODYiPjwvdXNlPgo8L2c+CjwvZz4KPGcg%0D%0AdHJhbnNmb3JtPSJ0cmFuc2xhdGUoNDYxNiwwKSI+CiA8dXNlIHhsaW5rOmhyZWY9IiNFMS1NSlNa%0D%0AMi0yMjExIiB4PSIwIiB5PSIwIj48L3VzZT4KPGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMTQ3LC0x%0D%0AMDkwKSI+CiA8dXNlIHRyYW5zZm9ybT0ic2NhbGUoMC43MDcpIiB4bGluazpocmVmPSIjRTEtTUpN%0D%0AQVRISS02OSIgeD0iMCIgeT0iMCI+PC91c2U+CiA8dXNlIHRyYW5zZm9ybT0ic2NhbGUoMC43MDcp%0D%0AIiB4bGluazpocmVmPSIjRTEtTUpNQUlOLTNEIiB4PSIzNDUiIHk9IjAiPjwvdXNlPgogPHVzZSB0%0D%0AcmFuc2Zvcm09InNjYWxlKDAuNzA3KSIgeGxpbms6aHJlZj0iI0UxLU1KTUFJTi0zMSIgeD0iMTEy%0D%0ANCIgeT0iMCI+PC91c2U+CjwvZz4KIDx1c2UgdHJhbnNmb3JtPSJzY2FsZSgwLjcwNykiIHhsaW5r%0D%0AOmhyZWY9IiNFMS1NSk1BVEhJLTZFIiB4PSI3MjEiIHk9IjE2MjciPjwvdXNlPgo8L2c+CiA8dXNl%0D%0AIHhsaW5rOmhyZWY9IiNFMS1NSk1BSU4tMjgiIHg9IjYwNjEiIHk9IjAiPjwvdXNlPgo8ZyB0cmFu%0D%0Ac2Zvcm09InRyYW5zbGF0ZSg2NDUwLDApIj4KIDx1c2UgeGxpbms6aHJlZj0iI0UxLU1KTUFUSEkt%0D%0ANTkiIHg9IjAiIHk9IjAiPjwvdXNlPgogPHVzZSB0cmFuc2Zvcm09InNjYWxlKDAuNzA3KSIgeGxp%0D%0Abms6aHJlZj0iI0UxLU1KTUFUSEktNjkiIHg9IjgyMiIgeT0iLTIxMyI+PC91c2U+CjwvZz4KIDx1%0D%0Ac2UgeGxpbms6aHJlZj0iI0UxLU1KTUFJTi0yMjEyIiB4PSI3NTk4IiB5PSIwIj48L3VzZT4KPGcg%0D%0AdHJhbnNmb3JtPSJ0cmFuc2xhdGUoODU5OSwwKSI+CiA8dXNlIHhsaW5rOmhyZWY9IiNFMS1NSk1B%0D%0AVEhJLTU5IiB4PSIwIiB5PSIwIj48L3VzZT4KIDx1c2UgdHJhbnNmb3JtPSJzY2FsZSgwLjcwNyki%0D%0AIHhsaW5rOmhyZWY9IiNFMS1NSk1BVEhJLTY5IiB4PSI4MjIiIHk9Ii0yMTMiPjwvdXNlPgogPHVz%0D%0AZSB4bGluazpocmVmPSIjRTEtTUpNQUlOLTVFIiB4PSIyMTIiIHk9IjI2MSI+PC91c2U+CjwvZz4K%0D%0APGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoOTUyNSwwKSI+CiA8dXNlIHhsaW5rOmhyZWY9IiNFMS1N%0D%0ASk1BSU4tMjkiIHg9IjAiIHk9IjAiPjwvdXNlPgogPHVzZSB0cmFuc2Zvcm09InNjYWxlKDAuNzA3%0D%0AKSIgeGxpbms6aHJlZj0iI0UxLU1KTUFJTi0zMiIgeD0iNTUwIiB5PSI1ODMiPjwvdXNlPgo8L2c+%0D%0ACiA8dXNlIHhsaW5rOmhyZWY9IiNFMS1NSk1BSU4tMkUiIHg9IjEwMzY4IiB5PSIwIj48L3VzZT4K%0D%0APC9nPgo8L3N2Zz4=)


It looks intimidating at first, but if you unpack the expression into a plain english set of steps it is pretty easy and intuitive.
    
1. For each data point, find the difference between the actual value (Yi) and the estimated value (Y^i)
2. Square the difference of each to get rid of negative values (so they don't cancel out positive values)
3. Add all of these square together and divide by the number of data points.